In [ ]:
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
import re
from docx import Document
import unicodedata

In [75]:
"""
####### NOT CORRECT #######
class DOCXProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.text = ""
        self.cleaned_text = ""
        self.extracted_info = {}
        self.tickable_info = {}

    # 1. Extracting raw text
    def extract_text(self):
        try:
            doc = Document(self.file_path)
        except Exception as e:
            print(f"Error reading DOCX file: {e}")
            return

        all_text = []

        for i, table in enumerate(doc.tables):
            for row in table.rows:
                for cell in row.cells:
                    cell_text = cell.text.strip()
                    if cell_text:
                        all_text.append(cell_text)

        self.text = "\n".join(all_text)

    # 2. Cleaning text (adjusting to keep important characters like punctuation and removing unnecessary spaces)
    def clean_text(self):
        text = unicodedata.normalize("NFKD", self.text)
        text = re.sub(r"[^a-zA-Z0-9.,!?%€$-:\n\s]", " ", text)  # Keep punctuation for dates, phone numbers, etc.
        text = text.lower()
        text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
        self.cleaned_text = text
        #print("Cleaned text:\n", self.cleaned_text, "\n")

    # 3. Extracting key-value and tickable options
    def extract_info(self):
        # Horrible Regex patterns (thanks chatGPT)
        tickable_patterns = {
            "gender": r"gender\s*[:\-]?\s*(male|female)\b",  # Match exactly 'male' or 'female'
            "pep": r"politically exposed person\s*[:\-]?\s*(no|yes)\b",
            "marital_status": r"marital status\s*[:\-]?\s*(divorced|married|single|widowed)\b",
            "employment_status": r"(?:current employment and function|employment status)\s*[:\-]?\s*(employee|self-employed|currently not employed)\b",
            "previous_profession": r"previous profession\s*[:\-]?\s*(retired|homemaker/housewife|student|diplomat|military representative|other|unknown)\b",
            "total_wealth": r"total wealth estimated\s*[:\-]?\s*(< eur 1\.5m|eur 1\.5m-5m|eur 5m-10m|eur 10m.-20m|eur 20m.-50m|> eur 50m)\b",
            "origin_of_wealth": r"origin of wealth\s*[:\-]?\s*(employment|inheritance|business|investments|sale of real estate|retirement package|other)\b",
            "estimated_assets": r"estimated assets\s*[:\-]?\s*(real estate|business|investments|deposits|equity|fixed income|structured products|alternative investments|investment funds|insurance|other investments)\b",
            "estimated_income": r"estimated total income\s*[:\-]?\s*(< eur 250,000|eur 250,000 - 500,000|eur 500,000 – 1m|> eur 1m)\b",
            "commercial_account": r"commercial account\s*[:\-]?\s*(yes|no)\b",
            "investment_risk_profile": r"investment risk profile\s*[:\-]?\s*(low|moderate|considerable|high)\b",
            "mandate_type": r"type of mandate\s*[:\-]?\s*(advisory|discretionary)\b",
            "investment_experience": r"investment experience\s*[:\-]?\s*(inexperienced|experienced|expert)\b",
            "investment_horizon": r"investment horizon\s*[:\-]?\s*(short|medium|long-term)\b",
}

        personal_info_patterns = {
            "last_name": r"last name\s*([\w\-]+)(?=\s*(first/ middle name \(s\)|address|country of domicile|date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "first_middle_name": r"first/ middle name \(s\)\s*([\w\s]+)(?=\s*(address|country of domicile|date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "address": r"address\s*([\w\s,.-]+?)(?=\s*(country of domicile|date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "country_of_domicile": r"country of domicile\s*([\w\s]+)(?=\s*(date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "date_of_birth": r"date of birth\s*(\d{4}-\d{2}-\d{2})(?=\s*(nationality|passport no|id type|id issue date|id expiry date))",
            "nationality": r"nationality\s*([\w\s]+)(?=\s*(passport no|id type|id issue date|id expiry date))",
            "passport_no": r"passport no/ unique id\s*([\w\d]+)(?=\s*(id type|id issue date|id expiry date))",
            "id_type": r"id type\s*([\w\s]+)(?=\s*(id issue date|id expiry date))",
            "id_issue_date": r"id issue date\s*(\d{4}-\d{2}-\d{2})(?=\s*id expiry date)",
            "id_expiry_date": r"id expiry date\s*(\d{4}-\d{2}-\d{2})",
            "telephone": r"telephone\s*(\+[\d\s]+)(?=\s*(e-mail|email))",
            "email": r"(e[-\s]*mail|email)\s*([\w\.-]+@[\w\.-]+)(?=\s*(telephone|country of domicile|date of birth|nationality|passport no|id type))",
        }

        # Extracting tickable fields
        tickable_results = {}
        for key, pattern in tickable_patterns.items():
            match = re.search(pattern, self.cleaned_text)
            if match:
                tickable_results[key] = match.group(1)
                #print(f"  Tickable field '{key}': {match.group(1)}")

        # Extracting personal information fields
        personal_info = {}
        for key, pattern in personal_info_patterns.items():
            match = re.search(pattern, self.cleaned_text)
            if match:
                if key == "first_middle_name":
                    full_name = match.group(1).strip()
                    names = full_name.split()
                    personal_info["first_name"] = names[0]
                    personal_info["middle_name"] = " ".join(names[1:]) if len(names) > 1 else ""
                else:
                    personal_info[key] = match.group(1)
                #print(f"  Personal info '{key}': {personal_info.get(key, '')}")

        # Combine personal info with other extracted information
        self.extracted_info = {**personal_info, **tickable_results}

        #print("\nExtracted Key-Value info:")
        #print(self.extracted_info)

        #print("\nExtracted tickable pptions:")
        #print(tickable_results)

        self.tickable_info = tickable_results

    # Run all steps
    def run_pipeline(self):
        self.extract_text()
        self.clean_text()
        self.extract_info()
        return {
            "text": self.text,
            "cleaned_text": self.cleaned_text,
            "info": self.extracted_info,
            "tickables": self.tickable_info
        }

# Run the full thing
if __name__ == "__main__":
    file_path = "/content/client_1/profile (2).docx"  # Update this path!!
    processor = DOCXProcessor(file_path)
    result = processor.run_pipeline()

    print("\nFinal text dict:")
    print(result["info"])

    print("\nFinal tickable fields dict:")
    print(result["tickables"])

    # There's a comment field I didn't extract.
    # TO DO in classification: check mandatory fields are filled or ticked
"""


Final text dict:
{'last_name': 'sanders', 'first_name': 'simon', 'middle_name': 'samuel', 'address': 'rue de la loi 38, 2999 namur', 'country_of_domicile': 'belgium ', 'date_of_birth': '1967-10-25', 'nationality': 'belgian ', 'passport_no': 'ql3957717', 'id_type': 'passport ', 'id_issue_date': '2021-03-04', 'id_expiry_date': '2028-03-03', 'telephone': '+32 0454 934 076 ', 'gender': 'female', 'marital_status': 'divorced', 'employment_status': 'employee', 'previous_profession': 'retired', 'origin_of_wealth': 'employment', 'estimated_assets': 'real estate', 'commercial_account': 'yes', 'investment_risk_profile': 'low', 'mandate_type': 'advisory', 'investment_experience': 'inexperienced', 'investment_horizon': 'short'}

Final tickable fields dict:
{'gender': 'female', 'marital_status': 'divorced', 'employment_status': 'employee', 'previous_profession': 'retired', 'origin_of_wealth': 'employment', 'estimated_assets': 'real estate', 'commercial_account': 'yes', 'investment_risk_profile': 'l

## CHECKBOXES

In [69]:

def extract_docx_text_with_checkboxes(file_path):
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"Error reading DOCX file: {e}")
        return ""

    all_text = []

    for i, table in enumerate(doc.tables):
        for row in table.rows:
            row_text = []
            for cell in row.cells:
                cell_runs = []
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        cell_runs.append(run.text)
                full_cell_text = "".join(cell_runs).strip()
                if full_cell_text:
                    row_text.append(full_cell_text)
            if row_text:
                all_text.append(" | ".join(row_text))

    return "\n".join(all_text)

file_path = "/content/client_1/profile (2).docx"  # update your path
extracted_text = extract_docx_text_with_checkboxes(file_path)
extracted_text

'Client Information\nLast Name | Sanders\nFirst/ Middle Name (s) | Simon Samuel\nAddress | Rue de la Loi 38, 2999 Namur\nCountry of Domicile | Belgium\nDate of birth | 1967-10-25\nNationality | Belgian\nPassport No/ Unique ID | QL3957717\nID Type | passport\nID Issue Date | 2021-03-04\nID Expiry Date | 2028-03-03\nGender | ☐ Female\t\t☒ Male\nAccount Holder – Contact Management and Services – Contact Info\nCommunication Medium | Telephone\t+32 0454 934 076\nE-Mail\tsimon.sanders@voo.be\nAccount Holder – Personal Info\nIs the client or associated person a Politically Exposed Person as defined in the Client Acceptance Policy? | ☒ No\t☐ Yes | If ‘Yes’, please complete the Approval Form (PEP) and ensure that appropriate approvals are obtained\nMarital Status | ☒ Divorced\t☐ Married\t☐ Single\t☐ Widowed | Fill out in case of a private individual\nHighest education attained | Tertiary\nEducation History | Université libre de Bruxelles (1990), Hasselt University (1995)\nAccount Holder – Profe

In [72]:

def extract_checked_options_from_text(text):
    checkbox_info = {}

    lines = text.split("\n")
    for line in lines:
        if "☒" in line:  # only process lines that have at least one checked box
            parts = line.split("|")
            label = parts[0].strip().lower() if len(parts) > 1 else "unknown"
            options_text = " | ".join(parts[1:]) if len(parts) > 1 else parts[0]

            # Find all (☐ or ☒) followed by the option text (up to tab, pipe, or end of line)
            matches = re.findall(r"(☒|☐)\s*([^\t|☐☒]+)", options_text)

            # Get only the options with ☒
            checked = [opt.strip().lower() for mark, opt in matches if mark == "☒"]

            if checked:
                checkbox_info[label] = checked if len(checked) > 1 else checked[0]

    return checkbox_info

docx_text = extract_docx_text_with_checkboxes("/content/client_1/profile (2).docx")
checkbox_data = extract_checked_options_from_text(docx_text)
checkbox_data


{'gender': 'male',
 'is the client or associated person a politically exposed person as defined in the client acceptance policy?': 'no',
 'marital status': 'divorced',
 'current employment and function': 'employee',
 'total wealth estimated': 'eur 1.5m-5m',
 'estimated assets': ['real estate', 'business'],
 'estimated total income p.a.': 'eur 250,000 - 500,000',
 'commercial account': 'no',
 'investment risk profile': 'low',
 'type of mandate': 'advisory',
 'investment experience': 'experienced',
 'investment horizon': 'short'}

# USE THIS VERSION OF THE PARSER PLEASE

In [83]:
import re
import unicodedata
from docx import Document

# Function to extract the raw text with checkboxes
def extract_docx_text_with_checkboxes(file_path):
    try:
        doc = Document(file_path)
    except Exception as e:
        print(f"Error reading DOCX file: {e}")
        return ""

    all_text = []

    for i, table in enumerate(doc.tables):
        for row in table.rows:
            row_text = []
            for cell in row.cells:
                cell_runs = []
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        cell_runs.append(run.text)
                full_cell_text = "".join(cell_runs).strip()
                if full_cell_text:
                    row_text.append(full_cell_text)
            if row_text:
                all_text.append(" | ".join(row_text))

    return "\n".join(all_text)

# Function to extract the checked options from the text
def extract_checked_options_from_text(text):
    checkbox_info = {}

    lines = text.split("\n")
    for line in lines:
        if "☒" in line:
            parts = line.split("|")
            label = parts[0].strip().lower() if len(parts) > 1 else "unknown"
            options_text = " | ".join(parts[1:]) if len(parts) > 1 else parts[0]

            matches = re.findall(r"(☒|☐)\s*([^\t|☐☒]+)", options_text)

            checked = [opt.strip().lower() for mark, opt in matches if mark == "☒"]

            if checked:
                checkbox_info[label] = checked if len(checked) > 1 else checked[0]

    return checkbox_info

# Main class to process DOCX files
class DOCXProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.text = ""
        self.cleaned_text = ""
        self.personal_info = {}
        self.tickable_info = {}
        self.communication_info = {}

    # 1. Extract raw text
    def extract_text(self):
        self.text = extract_docx_text_with_checkboxes(self.file_path)

    # 2. Clean the extracted text
    def clean_text(self):
        text = unicodedata.normalize("NFKD", self.text)
        text = re.sub(r"[^a-zA-Z0-9☐☒.,!?%€$-:\n\s@()]", " ", text)  # preserve checkboxes and special chars
        text = text.lower()
        text = re.sub(r"\s+", " ", text).strip()
        self.cleaned_text = text

    # 3. Extract text input and checked options
    def extract_info(self):
        personal_info_patterns = {
            "last_name": r"last name\s*([\w\-]+)(?=\s*(first/ middle name \(s\)|address|country of domicile|date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "first_name": r"first/ middle name \(s\)\s*([\w\s]+)(?=\s*(address|country of domicile|date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "address": r"address\s*([\w\s,.-]+?)(?=\s*(country of domicile|date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "country_of_domicile": r"country of domicile\s*([\w\s]+)(?=\s*(date of birth|nationality|passport no|id type|id issue date|id expiry date))",
            "date_of_birth": r"date of birth\s*(\d{4}-\d{2}-\d{2})(?=\s*(nationality|passport no|id type|id issue date|id expiry date))",
            "nationality": r"nationality\s*([\w\s]+)(?=\s*(passport no|id type|id issue date|id expiry date))",
            "passport_no": r"passport no/ unique id\s*([\w\d]+)(?=\s*(id type|id issue date|id expiry date))",
            "id_type": r"id type\s*([\w\s]+)(?=\s*(id issue date|id expiry date))",
            "id_issue_date": r"id issue date\s*(\d{4}-\d{2}-\d{2})(?=\s*id expiry date)",
            "id_expiry_date": r"id expiry date\s*(\d{4}-\d{2}-\d{2})",
            "telephone": r"telephone\s*(\+[\d\s]+)(?=\s*(e-mail|email))",
            "email": r"(e[-\s]*mail|email)\s*([\w\.-]+@[\w\.-]+)(?=\s*(telephone|country of domicile|date of birth|nationality|passport no|id type))",
        }

        # Extract personal information
        for key, pattern in personal_info_patterns.items():
            match = re.search(pattern, self.cleaned_text)
            if match:
                if key == "first_name":
                    full_name = match.group(1).strip()
                    names = full_name.split()
                    self.personal_info["first_name"] = names[0]
                    self.personal_info["middle_name"] = " ".join(names[1:]) if len(names) > 1 else ""
                else:
                    self.personal_info[key] = match.group(1)

        # Extract tickable fields
        self.tickable_info = extract_checked_options_from_text(self.text)

    # Extract communication fields (telephone, email)
    def extract_communication_info(self):
        communication_patterns = {
            "telephone": r"telephone\s*([\+0-9\s\-\(\)]+)(?=\s*(e[-\s]*mail|email))",
            "email": r"(e[-\s]*mail|email)\s*([\w\.-]+@[\w\.-]+)"
        }

        # Extract communication information
        for key, pattern in communication_patterns.items():
            match = re.search(pattern, self.cleaned_text)
            if match:
                self.communication_info[key] = match.group(2).strip()  # Adjusted to capture the email address correctly

    # Run the full pipeline
    def run_pipeline(self):
        self.extract_text()
        self.clean_text()
        self.extract_info()
        self.extract_communication_info()

        # Combine personal info with tickable options
        final_info = {**self.personal_info, **self.tickable_info, **self.communication_info}

        return {
            "text": final_info,
            "cleaned_text": self.cleaned_text,
            "info": final_info,
            "tickables": self.tickable_info
        }

# Example usage
if __name__ == "__main__":
    file_path = "/content/client_1/profile (2).docx"  # Update with actual file path
    processor = DOCXProcessor(file_path)
    result = processor.run_pipeline()

    print("\nFinal text dict:")
    print(result["text"])

    print("\nFinal tickable fields dict:")
    print(result["tickables"])



Final text dict:
{'last_name': 'sanders', 'first_name': 'simon', 'middle_name': 'samuel', 'address': 'rue de la loi 38, 2999 namur', 'country_of_domicile': 'belgium ', 'date_of_birth': '1967-10-25', 'nationality': 'belgian ', 'passport_no': 'ql3957717', 'id_type': 'passport ', 'id_issue_date': '2021-03-04', 'id_expiry_date': '2028-03-03', 'telephone': 'e-mail', 'gender': 'male', 'is the client or associated person a politically exposed person as defined in the client acceptance policy?': 'no', 'marital status': 'divorced', 'current employment and function': 'employee', 'total wealth estimated': 'eur 1.5m-5m', 'estimated assets': ['real estate', 'business'], 'estimated total income p.a.': 'eur 250,000 - 500,000', 'commercial account': 'no', 'investment risk profile': 'low', 'type of mandate': 'advisory', 'investment experience': 'experienced', 'investment horizon': 'short', 'email': 'simon.sanders@voo.be'}

Final tickable fields dict:
{'gender': 'male', 'is the client or associated per